<a href="https://colab.research.google.com/github/Musu1/Atlan_assignment_submission/blob/master/submission_atlan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install overpy

     |████████████████████████████████| 51kB 2.6MB/s 
  Created wheel for overpy: filename=overpy-0.4-cp36-none-any.whl size=45771 sha256=4b5a846c7d4586058f51ad62e9a87901516b283ffadd4dbf2593988254f5dc81
  Stored in directory: /root/.cache/pip/wheels/10/53/c2/e6b6f97e7bb419193bd3aafbe38628666f0f93a1cec9dc521d
Successfully built overpy


In [0]:
import overpy

In [0]:
api=overpy.Overpass()

# Important URLs
1. Tag Info API - https://taginfo.openstreetmap.org/tags
2. Overpass API documentation - https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL
3. Overpass API with query examples - https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example
4. Loading data from OSM with python - https://towardsdatascience.com/loading-data-from-openstreetmap-with-python-and-the-overpass-api-513882a27fd0
5. All the amenities list - https://wiki.openstreetmap.org/wiki/Key:amenity
6. Overpass API - http://overpass-turbo.eu/
7. Amazing documentation - https://buildmedia.readthedocs.org/media/pdf/python-overpy/latest/python-overpy.pdf

In [0]:
r = api.query("""
area["ISO3166-1"="Mumbai"][admin_level=2];
(node["amenity"="parking"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
""")

In [0]:
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

In [23]:
coords

[]

In [26]:
r = api.query("""
/*
This has been generated by the overpass-turbo wizard.
The original search was:
“amenity=* and name=*”
*/
[out:csv(amenity, ::lat, ::lon)][timeout:25];
// gather results
(
  area[name="Mumbai"];
  // query part for: “amenity=* and name=*”
  node["amenity"]["name"](area);
);
// print results
out body;
>;
out skel qt;
""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OverpassUnknownContentType: ignored